### 如何加载使用tool
- 加载预制tool的方法
- 几种tool的使用方式
---
langchain预制了大量的tools，基本这些工具都能满足大部分需求  
https://github.com/langchain-ai/langchain/tree/master/docs/docs/integrations/tools

In [ ]:
# 添加预制工具的方法很简单
from langchain.agents import load_tools
tool_names=[...]
tools=load_tools(tool_names) # 使用load方法

# 有些tool需要单独设置llm
from langchain.agents import load_tools
tool_names=[...]
llm=...
tools=load_tools(tool_names,llm=llm)

#### SerpApi
- 最常见的聚合搜索引擎  
https://serpapi.com/dashboard

In [ ]:
# 提前在环境变量中设置SERPAPI_API_KEY
from langchain.utilities import SerpAPIWrapper

search=SerpAPIWrapper()

search.run("...?")

In [ ]:
# 支持自定义参数，比如 将引擎切换到bing，设置搜索语言等
params={
    "engine":"bing",    # 搜索引擎
    "gl":"us",          # 地理位置
    "hl":"en"           # 界面语言
}

search=SerpAPIWrapper(params=params)

search.run("...?")

#### Dall-E
- Dall-E是openai出品的文到图AI大模型

In [1]:
! pip install opencv-python scikit-image

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.8/39.5 MB 4.2 MB/s eta 0:00:10
   - -

In [5]:
from langchain.chat_models.tongyi import ChatTongyi

llm=ChatTongyi(model="qwen-plus",temperature=0.8,max_token=1000)

In [8]:
from langchain.agents import initialize_agent,load_tools,AgentType

tools=load_tools(["dalle-image-generator"])

agent=initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

output=agent.run("生成一辆宝马车")



> Entering new AgentExecutor chain...
我需要生成一辆宝马车的图像，为此我需要使用Dall-E图像生成器。
Action: Dall-E-Image-Generator
Action Input: 一辆宝马车

InvalidRequestError: <empty message>

#### Eleven Labs Text2Speech
- ElevenLabs是非常优秀的TTS合成API

In [12]:
! pip install elevenlabs

In [13]:
! pip install --upgrade pydantic

In [1]:
from langchain_community.tools import ElevenLabsText2SpeechTool

text2speak="我是你爸真伟大，养你这么大"

tts=ElevenLabsText2SpeechTool(
    voice="Bella",
    text_to_speak=text2speak,
    verbose=True
)
tts.name

'eleven_labs_text2speech'

In [5]:
speech_file=tts.run(text2speak)

tts.play(speech_file) # 按完整文件输出
tts.stream_speech(text2speak) # 边生成边输出

RuntimeError: Error while running ElevenLabsText2SpeechTool: a bytes-like object is required, not 'generator'

#### GraphQL
- 一种api查询语言，类似sql，我们用它来查询奈飞的数据库，查找一下和星球大战相关的电影  

In [9]:
! pip install httpx

In [10]:
! pip install gql


   ---------------------------------------- 0/2 [graphql-core]
   ---------------------------------------- 0/2 [graphql-core]
   -------------------- ------------------- 1/2 [gql]
   ---------------------------------------- 2/2 [gql]



In [7]:
! pip install requests_toolbelt

In [22]:
from langchain.chat_models.tongyi import ChatTongyi
from langchain.agents import load_tools,initialize_agent,AgentType
from langchain.utilities import GraphQLAPIWrapper

llm=ChatTongyi(model="qwen-plus",temperature=0.5,max_token=1000)

tools=load_tools(
    ["graphql"],
    graphql_endpoint="https://swapi-graphql.netlify.app/.netlify/functions/index",
    fetch_schema_from_transport=False
)

agent=initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [25]:
graghql_fields = """
query {
  allFilms {
    films {
      title
      director
      releaseDate
      speciesConnection {
        species {
          name
          classification
          homeworld {
            name
          }
        }
      }
    }
  }
}

"""

suffix="Search for the titles of all the starwars films stored in the graphql database that has this schema,and answer in Chinese."

full_query = suffix + graghql_fields

agent.run(full_query)



> Entering new AgentExecutor chain...
我需要查询GraphQL数据库以获取所有《星球大战》电影的标题，并将结果翻译成中文。
Action: query_graphql
Action Input: query { allFilms { films { title } } }

TransportQueryError: {'message': 'Missing query'}

: 